In [1]:
import streamlit as st
import pandas as pd
import numpy as np

# Predefined list of ingredients (these are now column names in your dataset)
ingredients = [
    'almond', 'amaretto', 'anchovy', 'anise', 'apple', 'apple juice', 'apricot', 
    'artichoke', 'arugula', 'asian pear', 'asparagus', 'avocado', 'bacon', 'banana', 
    'barley', 'basil', 'bass', 'bean', 'beef', 'beef rib', 'beef shank', 'beef tenderloin', 
    'beer', 'beet', 'bell pepper', 'berry', 'biscuit', 'blackberry', 'blue cheese', 
    'blueberry', 'bok choy', 'bourbon', 'bran', 'brandy', 'bread', 'breadcrumbs', 'brie', 
    'broccoli', 'broccoli rabe', 'brown rice', 'brussel sprout', 'bulgur', 'butter', 
    'buttermilk', 'butternut squash', 'cabbage', 'calvados', 'campari', 'cantaloupe', 
    'capers', 'caraway', 'cardamom', 'carrot', 'cashew', 'cauliflower', 'caviar', 'celery', 
    'chambord', 'champagne', 'chard', 'chartreuse', 'cheddar', 'cheese', 'cherry', 
    'chestnut', 'chicken', 'chickpea', 'chile pepper', 'chive', 'chocolate', 'cilantro', 
    'cinnamon', 'clam', 'clove', 'coconut', 'cod', 'coffee', 'collard greens', 
    'corn', 'cornmeal', 'cottage cheese', 'couscous', 'crab', 'cranberry', 'cream cheese', 
    'cucumber', 'cumin', 'currant', 'date', 'dill', 'duck', 'egg', 'eggplant', 'endive', 
    'escarole', 'fennel', 'feta', 'fig', 'fish', 'fontina', 'garlic', 'gin', 'ginger', 
    'goat cheese', 'goose', 'gouda', 'grape', 'grapefruit', 'green bean', 
    'ground beef', 'ground lamb', 'guava', 'halibut', 'ham', 'hazelnut', 'honeydew', 
    'horseradish', 'hot pepper', 'jalapeño', 'kale', 'kiwi', 'kumquat', 'lamb', 'lamb chop', 
    'lamb shank', 'leek', 'lemon', 'lemon juice', 'lemongrass', 'lentil', 'lettuce', 
    'lima bean', 'lime', 'lime juice', 'lingonberry', 'lobster', 'lychee', 'macadamia nut', 
    'mango', 'marsala', 'mayonnaise', 'mint', 'molasses', 
    'monterey jack', 'mozzarella', 'mushroom', 'mussel', 'mustard', 'mustard greens', 
    'nectarine', 'noodle', 'nutmeg', 'oat', 'oatmeal', 'octopus', 'okra', 'olive', 'onion', 
    'orange', 'orange juice', 'oregano', 'oyster', 'papaya', 'paprika', 'parmesan', 'parsley', 
    'parsnip', 'pasta', 'pea', 'peach', 'peanut', 'peanut butter', 'pear', 'pecan', 'pepper', 
    'pernod', 'persimmon', 'pineapple', 'pistachio', 'plantain', 'plum', 'pomegranate', 
    'pomegranate juice', 'pork', 'pork chop', 'pork rib', 'pork tenderloin', 'port', 'potato', 
    'prosciutto', 'prune', 'pumpkin', 'quail', 'quinoa', 'rabbit', 'radicchio', 'radish', 
    'raisin', 'raspberry', 'red wine', 'rhubarb', 'rice', 'ricotta', 'rosemary', 'rum', 
    'rutabaga', 'rye', 'saffron', 'sage', 'sake', 'salmon', 'sausage', 'scallop', 'sesame', 
    'sesame oil', 'shallot', 'shrimp', 'sour cream', 'soy sauce', 'spinach', 
    'squash', 'squid', 'steak', 'strawberry', 'sugar snap pea', 'swiss cheese', 
    'swordfish', 'tamarind', 'tangerine', 'tapioca', 'tarragon', 'tequila', 'thyme', 'tilapia', 
    'tofu', 'tomatillo', 'tomato', 'trout', 'tuna', 'turkey', 'turnip', 'vanilla', 'veal', 
    'vinegar', 'vodka', 'walnut', 'watercress', 'watermelon', 'whiskey', 'white wine', 
    'whole wheat', 'wild rice', 'wine', 'yogurt', 'zucchini'
]

# Load the data
@st.cache_data
def load_data():
    return pd.read_csv('final_data.csv')  # Replace with your actual file path

final_data = load_data()

# Function to get recipe suggestions
def get_recipe_suggestions(pantry_ingredients, n=5):
    # Calculate match score based on ingredients present in the recipe
    match_scores = final_data[pantry_ingredients].sum(axis=1) / final_data[ingredients].sum(axis=1)
    
    # Sort recipes by match score and return top n
    return final_data.assign(match_score=match_scores).sort_values('match_score', ascending=False).head(n)

# Streamlit app
st.title('🍽️ Pantry Recipe Suggester')

st.write("""
Welcome to the Pantry Recipe Suggester! 
Input the ingredients you have in your pantry, and we'll suggest recipes you can make.
""")

# Multi-select for ingredients
pantry_ingredients = st.multiselect(
    'Select ingredients you have in your pantry:',
    ingredients
)

# Button to get suggestions
if st.button('Get Recipe Suggestions'):
    if pantry_ingredients:
        suggestions = get_recipe_suggestions(pantry_ingredients)
        
        st.subheader('📝 Here are your recipe suggestions:')
        for i, (_, recipe) in enumerate(suggestions.iterrows(), 1):
            with st.expander(f"{i}. {recipe['title']} (Match: {recipe['match_score']:.2%})"):
                st.write(f"**Rating:** {recipe['rating']:.2f} ⭐")
                st.write(f"**Calories:** {recipe['calories_rf']:.0f}")
                st.write(f"**Protein:** {recipe['protein_rf']:.1f}g")
                st.write(f"**Fat:** {recipe['fat_rf']:.1f}g")
                st.write(f"**Sodium:** {recipe['sodium_rf']:.0f}mg")
                
                st.write("**Ingredients:**")
                recipe_ingredients = [ing for ing in ingredients if recipe[ing] == 1]
                for ingredient in recipe_ingredients:
                    if ingredient in pantry_ingredients:
                        st.markdown(f"- ✅ {ingredient}")
                    else:
                        st.markdown(f"- {ingredient}")
                
                if 'instructions' in recipe:
                    st.write("**Instructions:**")
                    st.write(recipe['instructions'])
                else:
                    st.write("Instructions not available for this recipe.")
    else:
        st.warning('Please select at least one ingredient from your pantry.')

# Additional features
st.sidebar.header('📊 App Statistics')
st.sidebar.write(f"Total Recipes: {len(final_data)}")
st.sidebar.write(f"Total Ingredients: {len(ingredients)}")

# Most common ingredients
ingredient_counts = final_data[ingredients].sum().sort_values(ascending=False)
st.sidebar.subheader('🥕 Top 10 Common Ingredients')
st.sidebar.table(ingredient_counts.head(10))

# Footer
st.markdown("""
---
Created with ❤️ using Streamlit | Data source: Recipe dataset
""")

C:\Users\user\anaconda3\Lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (
2024-10-05 18:40:34.937 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager
2024-10-05 18:40:35.593 
  command:

    streamlit run C:\Users\user\anaconda3\Lib\site-packages\ipykernel_launcher.py [ARGUMENTS]
2024-10-05 18:40:35.595 No runtime found, using MemoryCacheStorageManager


DeltaGenerator()